In [1]:
# suppress warnings
import warnings
warnings.filterwarnings("ignore")

# autoload modules
%load_ext autoreload
%autoreload 2

In [2]:
# Data analysis libraries
import pandas as pd
import margin_analysis as ma

In [3]:
# get customer and related data
customers = pd.read_csv("data/active_customers.csv")

# change customer_id to string
customers["customer_id"] = customers["customer_id"].astype(str)

In [4]:
augmented_line_items = pd.read_csv("data/augmented_line_items.csv")
augmented_line_items["created_date"] = pd.to_datetime(augmented_line_items["created_date"], errors="coerce")
len(augmented_line_items)

411008

In [5]:
line_itmes_for_analysis = augmented_line_items.copy()

In [6]:
line_itmes_for_analysis["total_amount"].sum()

np.float64(671916686.6898111)

In [7]:
grouping_columns = ["subsidiary_name", "location", "sku", "item_name", "display_name", "item_type", "manufacturer"]
missing_counts = line_itmes_for_analysis[grouping_columns].isna().sum()
missing_counts

subsidiary_name         0
location                0
sku                     0
item_name               0
display_name       193862
item_type               0
manufacturer            0
dtype: int64

In [8]:
line_itmes_for_analysis[grouping_columns] = line_itmes_for_analysis[grouping_columns].fillna("Not Assigned")

In [9]:
grouping_columns=["sku", "item_name", "display_name", "item_type", "manufacturer"]
total_booked_sales_by_sku = ma.calculate_total_booked_sales(line_itmes_for_analysis, grouping_columns=grouping_columns)
total_booked_sales_by_sku["total_booked_sales"].sum()

np.float64(671916686.6898112)

In [10]:
# remove rows with zero quantity
line_itmes_for_analysis = line_itmes_for_analysis[line_itmes_for_analysis["quantity"] != 0]
line_itmes_for_analysis["total_amount"].sum()

np.float64(671916686.6898115)

In [11]:
# remove rows with both zero total_amount and zero total_cost
line_itmes_for_analysis = line_itmes_for_analysis[~((line_itmes_for_analysis["total_amount"] == 0) & (line_itmes_for_analysis["total_cost"] == 0))]
line_itmes_for_analysis["total_amount"].sum()

np.float64(671916686.6898112)

In [12]:
# remove rows with gross profit percent less than -100%
line_itmes_for_analysis = line_itmes_for_analysis[line_itmes_for_analysis["gross_profit_percent"] >= -1.5]
line_itmes_for_analysis["total_amount"].sum()

np.float64(671713982.3398114)

In [13]:
grouping_columns = ["subsidiary_name", "location", "sku", "item_name", "display_name", "item_type", "manufacturer"]
top_booked_material_skus = ma.calculate_total_booked_sales(line_itmes_for_analysis, grouping_columns)
top_booked_material_skus["total_booked_sales"].sum()

np.float64(671713982.3398113)

In [14]:
top_skus_by_subsidiary = top_booked_material_skus.groupby("subsidiary_name").apply(lambda x: x.nlargest(5000, "total_booked_sales"), include_groups=True).reset_index(drop=True)
top_skus_by_subsidiary["total_booked_sales"].sum()

np.float64(554187130.3450187)

In [16]:
# print formatted % of top_booked_material_skus included in top_skus_by_subsidiary
print(f"{top_skus_by_subsidiary['total_booked_sales'].sum() / top_booked_material_skus['total_booked_sales'].sum():.2%}")

82.50%


In [17]:
# calculate monthy sales by sku by subsidiary
grouping_columns = ["created_date", "subsidiary_name", "location", "company_name", "end_market", "sku", "item_name", "display_name", "item_type", "manufacturer"]
monthly_booked_sales_by_sku_by_subsidiary = ma.calculate_total_booked_sales(line_itmes_for_analysis, grouping_columns)
monthly_booked_sales_by_sku_by_subsidiary["total_booked_sales"].sum()

np.float64(671713982.3398112)

In [18]:
# monthly_booked_sales_by_sku_by_subsidiary.to_excel("data/monthly_booked_sales_by_sku_by_subsidiary.xlsx", index=False)

In [19]:
monthly_booked_sales_by_sku_by_subsidiary

,time_period,subsidiary_name,location,company_name,end_market,sku,item_name,display_name,item_type,manufacturer,avg_cost,avg_unit_price,total_booked_sales,total_gross_profit,order_count,qty_sold,avg_gross_profit_per_unit,avg_gross_margin_pct
0,2022-01,Allied Instrumentation,Appleton,Ag Processing Inc Algona,Biofuels,5428,021114020-686-0000,DIAPHRAGM 140 SQ IN ACTUATOR,Inventory Item,Not Specified,93.31,185.00,185.00,91.69,1,1.0,91.69,0.495622
1,2022-01,Allied Instrumentation,Appleton,"Ahlstrom - Mosinee, WI",Paper & Pulp,622010,AV FS AccuTest,Not Assigned,Service,Not Specified,0.00,965.00,965.00,965.00,1,1.0,965.00,1.000000
2,2022-01,Allied Instrumentation,Appleton,"Ahlstrom - Mosinee, WI",Paper & Pulp,622040,AV FS Mileage,Not Assigned,Service,Not Specified,0.56,1.50,285.00,178.60,1,190.0,0.94,0.626667
3,2022-01,Allied Instrumentation,Appleton,"Ahlstrom - Mosinee, WI",Paper & Pulp,622043,AV FS Per Diem,Not Assigned,Service,Not Specified,45.00,65.00,65.00,20.00,1,1.0,20.00,0.307692
4,2022-01,Allied Instrumentation,Appleton,"Ahlstrom - Mosinee, WI",Paper & Pulp,622054,AV FS Travel Time OT/SAT,Not Assigned,Service,Not Specified,48.50,150.00,600.00,406.00,1,4.0,101.50,0.676667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229144,2024-11,Valve Sales Inc.,Tulsa,"XTO Energy, Inc - Ardmore",Upstream O&G,315713,315713,Asahi Actuator Beacon Indicator,Inventory Item,Asahi America,10.40,17.50,17.50,7.10,1,1.0,7.10,0.405714
229145,2024-11,Valve Sales Inc.,Tulsa,"XTO Energy, Inc - Ardmore",Upstream O&G,317196,317196,Pinion Gear for Asahi,Inventory Item,Asahi America,13.04,21.73,21.73,8.69,1,1.0,8.69,0.399908
229146,2024-11,Valve Sales Inc.,Tulsa,"XTO Energy, Inc - Ardmore",Upstream O&G,323117,323117,Peaktronics DHC-400 Digital Card (UL Blue Dot),Inventory Item,Peaktronics,231.36,432.00,3888.00,1805.76,2,9.0,200.64,0.464444
229147,2024-11,Valve Sales Inc.,Tulsa,"XTO Energy, Inc - Ardmore",Upstream O&G,1696367,Shop Labor - UL Asahi Actuator Repair,Shop Labor - UL Asahi Actuator Repair,Service,Not Specified,82.50,165.00,660.00,330.00,4,4.0,82.50,0.500000


In [20]:
monthly_top_sku_sales_by_subsidiary = ma.filter_top_skus(monthly_booked_sales_by_sku_by_subsidiary, top_skus_by_subsidiary)
monthly_top_sku_sales_by_subsidiary["total_booked_sales"].sum()

np.float64(554195717.1050187)

In [21]:
monthly_top_sku_sales_by_subsidiary

,time_period,subsidiary_name,location,company_name,end_market,sku,item_name,display_name,item_type,manufacturer,avg_cost,avg_unit_price,total_booked_sales,total_gross_profit,order_count,qty_sold,avg_gross_profit_per_unit,avg_gross_margin_pct
0,2022-01,Allied Instrumentation,Appleton,"Ahlstrom - Mosinee, WI",Paper & Pulp,622010,AV FS AccuTest,Not Assigned,Service,Not Specified,0.00,965.0,965.0,965.00,1,1.0,965.00,1.000000
1,2022-01,Allied Instrumentation,Appleton,"Ahlstrom - Mosinee, WI",Paper & Pulp,622040,AV FS Mileage,Not Assigned,Service,Not Specified,0.56,1.5,285.0,178.60,1,190.0,0.94,0.626667
2,2022-01,Allied Instrumentation,Appleton,"Ahlstrom - Mosinee, WI",Paper & Pulp,622043,AV FS Per Diem,Not Assigned,Service,Not Specified,45.00,65.0,65.0,20.00,1,1.0,20.00,0.307692
3,2022-01,Allied Instrumentation,Appleton,"Ahlstrom - Mosinee, WI",Paper & Pulp,622054,AV FS Travel Time OT/SAT,Not Assigned,Service,Not Specified,48.50,150.0,600.0,406.00,1,4.0,101.50,0.676667
4,2022-01,Allied Instrumentation,Appleton,"Ahlstrom - Mosinee, WI",Paper & Pulp,622055,AV FS Travel Time STD,Not Assigned,Service,Not Specified,37.00,110.0,440.0,292.00,1,4.0,73.00,0.663636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125372,2024-11,Valve Sales Inc.,Tulsa,"XTO Energy, Inc - Ardmore",Upstream O&G,315315,315315,Asahi Actuator Beacon Globe,Inventory Item,Asahi America,12.60,21.0,42.0,16.80,2,2.0,8.40,0.400000
125373,2024-11,Valve Sales Inc.,Tulsa,"XTO Energy, Inc - Ardmore",Upstream O&G,315713,315713,Asahi Actuator Beacon Indicator,Inventory Item,Asahi America,10.40,17.5,17.5,7.10,1,1.0,7.10,0.405714
125374,2024-11,Valve Sales Inc.,Tulsa,"XTO Energy, Inc - Ardmore",Upstream O&G,323117,323117,Peaktronics DHC-400 Digital Card (UL Blue Dot),Inventory Item,Peaktronics,231.36,432.0,3888.0,1805.76,2,9.0,200.64,0.464444
125375,2024-11,Valve Sales Inc.,Tulsa,"XTO Energy, Inc - Ardmore",Upstream O&G,1696367,Shop Labor - UL Asahi Actuator Repair,Shop Labor - UL Asahi Actuator Repair,Service,Not Specified,82.50,165.0,660.0,330.00,4,4.0,82.50,0.500000


In [22]:
monthly_top_sku_sales_by_subsidiary.to_excel("data/monthly_top_sku_sales_by_subsidiary.xlsx", index=False)

In [ ]:
grouping_columns = ["created_date", "sku", "item_name", "display_name", "item_type", "manufacturer"]
annual_booked_sales_by_sku = ma.calculate_total_booked_sales(line_itmes_for_analysis, grouping_columns, time_period="year")

In [ ]:
annual_booked_sales_by_sku

In [ ]:
filled_monthly_top_sku_sales_by_subsidiary = ma.fill_missing_months(monthly_top_sku_sales_by_subsidiary)
filled_monthly_top_sku_sales_by_subsidiary["total_booked_sales"].sum()

In [ ]:
filled_monthly_top_sku_sales_by_subsidiary = ma.fill_missing_string_values(filled_monthly_top_sku_sales_by_subsidiary)
filled_monthly_top_sku_sales_by_subsidiary["total_booked_sales"].sum()

In [ ]:
ttm_margin_by_top_sku_by_subsidiary = ma.calculate_ttm_avg(filled_monthly_top_sku_sales_by_subsidiary)
ttm_margin_by_top_sku_by_subsidiary = ttm_margin_by_top_sku_by_subsidiary.fillna(0)
ttm_margin_by_top_sku_by_subsidiary["total_booked_sales"].sum()

In [ ]:
# calculate the change in margin for each sku each month
ttm_margin_by_top_sku_by_subsidiary["ttm_margin_change"] = ttm_margin_by_top_sku_by_subsidiary.groupby(["subsidiary_name", "sku"])["ttm_avg_gross_margin_percent"].diff()
ttm_margin_by_top_sku_by_subsidiary["ttm_margin_change"] = ttm_margin_by_top_sku_by_subsidiary["ttm_margin_change"].fillna(0)

In [ ]:
# calculate the percentage change in margin for each sku each month by using the ttm_avg_gross_margin_percent and margin_change columns
ttm_margin_by_top_sku_by_subsidiary["ttm_margin_change_pct"] = ttm_margin_by_top_sku_by_subsidiary["ttm_margin_change"] / ttm_margin_by_top_sku_by_subsidiary["ttm_avg_gross_margin_percent"]
ttm_margin_by_top_sku_by_subsidiary["ttm_margin_change_pct"] = ttm_margin_by_top_sku_by_subsidiary["ttm_margin_change_pct"].fillna(0)

In [ ]:
ttm_margin_by_top_sku_by_subsidiary